In [1]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
import json 
import re
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt 

import openai
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import style
style.use('ggplot')
from collections import Counter

# from __future__ import print_function
import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim

In [3]:
data = pd.read_csv("./data_clean.csv")
data = pd.DataFrame(data)

In [4]:
from dotenv import dotenv_values
import openai
env_vars = dotenv_values('.env')
openai.api_key = env_vars.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai.api_key)

In [ ]:
input="function add(a, b) { return a + b; }",
model="text-embedding-ada-002"

def get_embedding(text, model):
   #text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model)['data'][0]['embedding']

get_embedding(input, model)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

embeddings_2d = embed_model.embed_documents("function add(a, b) { return a + b; }")
len(embeddings_2d), len(embeddings_2d[0])

In [11]:
# Define a collection of code snippets
code_snippets = [
  """
    def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
  """,
  """
  def fibonacci(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 2:
        return 1
    else:
        result = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
        memo[n] = result
        return result
  """,
  """
    unsigned long long emi(unsigned int n) {
      if (n == 0 || n == 1) {
          return 1;
      } else {
          return n * emi(n - 1);
      }
    }
  """,
  """
    def looping(n):    
          print(n) 
  """,
]

embeddings=[]
for code in code_snippets:
    response = client.embeddings.create(input=code, model="text-embedding-3-small")
    embedding = response.data[0].embedding  # Access the 'embedding' attribute within the 'data' attribute
    embeddings.append(embedding)

# Generate embeddings for each code snippet
# embeddings = [client.embeddings.create(input=code, model="text-embedding-ada-002")['embedding'] for code in code_snippets]

# Convert the embeddings to a format suitable for cosine similarity
embeddings_matrix = [embedding for embedding in embeddings]

# Compute the cosine similarity between the embeddings
similarity_matrix = cosine_similarity(embeddings_matrix)

# Print the similarity matrix
print("Cosine Similarity Matrix:")
print(similarity_matrix)

Cosine Similarity Matrix:
[[1.         0.53986259 0.3341897  0.55777129]
 [0.53986259 1.         0.37462465 0.43740911]
 [0.3341897  0.37462465 1.         0.31579159]
 [0.55777129 0.43740911 0.31579159 1.        ]]


In [10]:
# from openai.embeddings_utils import get_embedding, cosine_similarity

# data['code_embedding'] = data['code_content'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

# def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
#    embedding = get_embedding(code_query, model='text-embedding-3-small')
#    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

#    res = df.sort_values('similarities', ascending=False).head(n)
#    return res
# res = search_functions(data, 'Completions API tests', n=3)

In [16]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load pre-trained CodeBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

# Example Python code snippets
code_snippets = [
    "def factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n - 1)",
    "def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n - 1) + fibonacci(n - 2)"
]

# Tokenize and encode the code snippets
encoded_inputs = tokenizer(code_snippets, padding=True, truncation=True, return_tensors="pt")

# Generate code embeddings
with torch.no_grad():
    outputs = model(**encoded_inputs)
    embeddings_bert = outputs.last_hidden_state.mean(dim=1)

# Print the code embeddings
for i, snippet in enumerate(code_snippets):
    print("Code Snippet:", snippet)
    print("Embedding:", embeddings_bert[i])
    print()

tokenizer_config.json: 100%|██████████| 25.0/25.0 [00:00<00:00, 13.4kB/s]
config.json: 100%|██████████| 498/498 [00:00<00:00, 1.40MB/s]
vocab.json: 100%|██████████| 899k/899k [00:04<00:00, 212kB/s]
merges.txt: 100%|██████████| 456k/456k [00:01<00:00, 283kB/s]
special_tokens_map.json: 100%|██████████| 150/150 [00:00<00:00, 568kB/s]
pytorch_model.bin: 100%|██████████| 499M/499M [02:29<00:00, 3.34MB/s] 


Code Snippet: def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)
Embedding: tensor([-2.7771e-01,  1.5763e-01,  2.9554e-01, -1.4439e-02, -2.2769e-01,
        -7.4887e-01, -3.6330e-02,  1.8782e-01,  5.6236e-01,  3.8150e-01,
        -2.9504e-01,  5.7466e-01, -3.3646e-01,  6.0460e-02,  7.8014e-01,
        -2.2050e-01,  2.0446e-01,  2.4504e-01, -8.1589e-02,  4.3083e-01,
        -1.5995e-01, -4.3422e-01,  5.6799e-01, -1.8546e-01,  5.5998e-01,
         3.4823e-01, -3.2295e-02,  8.2829e-01, -4.7591e-01,  5.8467e-01,
        -1.3403e-01, -1.9524e-02,  1.4167e+00,  1.7739e-01,  3.5241e-01,
        -2.5652e-01, -3.4333e-01,  2.7224e-01,  2.0292e-01, -2.4290e-01,
        -5.0367e-02,  7.0001e-01, -9.2676e-01, -2.6064e-01,  5.5162e-01,
         2.3583e-01,  4.3142e-01,  1.0256e-01, -3.9705e-02,  4.6246e-01,
         5.4635e-01,  2.8289e-01, -4.7521e-01, -2.1048e-01,  4.3169e-01,
         4.9456e-01, -1.0646e+00, -7.2688e-01, -1.3995e-01, -4.661

In [17]:
# Convert the embeddings to a format suitable for cosine similarity
embeddings_matrix = [embedding for embedding in embeddings]

# Compute the cosine similarity between the embeddings
similarity_matrix = cosine_similarity(embeddings_matrix)

tensor([[-0.2777,  0.1576,  0.2955,  ..., -0.2448, -0.3847,  0.6505],
        [-0.3416,  0.1528,  0.2126,  ..., -0.3933, -0.4310,  0.6557]])

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
documents = ["This is the first document.",
			"This document is the second document.",
			"And this is the third one.",
			"Is this the first document?"]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
feature_names = vectorizer.get_feature_names_out()

print("Bag-of-Words Matrix:")
print(X.toarray())
print("Vocabulary (Feature Names):", feature_names)


Bag-of-Words Matrix:
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
Vocabulary (Feature Names): ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'  # Change to the desired model, e.g., 'gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Define a prompt
prompt = "Once upon a time"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate output based on the prompt
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

/home/vboxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 665/665 [00:00<00:00, 2.44MB/s]
model.safetensors: 100%|██████████| 548M/548M [03:19<00:00, 2.75MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 57.9kB/s]
tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 90.4kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:01<00:00, 830kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.94MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 2.66MB/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys, json
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

import llm_model as utils

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load pre-trained CodeBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

# Example Python code snippets
code_snippets = [
    "def factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n - 1)",
    "def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n - 1) + fibonacci(n - 2)"
]

# Tokenize and encode the code snippets
encoded_inputs = tokenizer(code_snippets, padding=True, truncation=True, return_tensors="pt")

# Generate code embeddings
with torch.no_grad():
    outputs = model(**encoded_inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

# Print the code embeddings
for i, snippet in enumerate(code_snippets):
    print("Code Snippet:", snippet)
    print("Embedding:", embeddings[i])
    print()

In [ ]:
optimal_k = utils.silhouette_to_find_optimal_k(embeddings)

In [ ]:
cluster_labels, job_embeddings = utils.kmeans_clustering(optimal_k, embeddings)

In [ ]:
import numpy as np

def Fetch_Each_clustering_Content(texts, cluster_labels):
    # Write text data for each cluster to separate text files
    for cluster_label in range(optimal_k):
        cluster_texts = np.array(texts)[np.array(cluster_labels) == cluster_label]
        save_path = f'./txt/hon_{cluster_label}_texts.txt'

        with open(save_path, 'w') as file:
            for text in cluster_texts:
                file.write(text + '\n')
    
    return cluster_labels

# Specify the optimal number of clusters and the path for saving the text file
texts = data['code_content'].splitlines()
# Perform k-means clustering and save the cluster labels
cluster_labels = Fetch_Each_clustering_Content(texts, cluster_labels)